In [1]:
from pyspark import SparkContext
sc=SparkContext()

24/11/29 17:40:45 WARN Utils: Your hostname, Zhijie resolves to a loopback address: 127.0.1.1; using 10.27.109.93 instead (on interface wlo1)
24/11/29 17:40:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/29 17:40:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/11/29 17:40:57 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [75]:
data1 = sc.textFile('Phones_accelerometer.csv', 8)
data2 = sc.textFile('Phones_gyroscope.csv', 8)
data3 = sc.textFile('Watch_accelerometer.csv', 8)
data4 = sc.textFile('Watch_gyroscope.csv', 8)

In [76]:
def configure_data(line):
    datas = line.split(',')
    x = float(datas[3])
    y = float(datas[4])
    z = float(datas[5])
    key = (datas[6], datas[7], datas[9])
    value = (
        x, y, z,
        x**2, y**2, z**2,
        x, y, z,
        x, y, z,
        1
    )
    return  key, value

def reduce_data(a, b):
    sum_x = a[0] + b[0]
    sum_y = a[1] + b[1]
    sum_z = a[2] + b[2]
    sum_x2 = a[3] + b[3]
    sum_y2 = a[4] + b[4]
    sum_z2 = a[5] + b[5]
    min_x = min(a[6], b[6])
    min_y = min(a[7], b[7])
    min_z = min(a[8], b[8])
    max_x = max(a[9], b[9])
    max_y = max(a[10], b[10])
    max_z = max(a[11], b[11])
    count = a[12] + b[12]
    return (
        sum_x, sum_y, sum_z,
        sum_x2, sum_y2, sum_z2,
        min_x, min_y, min_z,
        max_x, max_y, max_z,
        count
    )

def calculate_data(data):
    key, value = data
    sum_x, sum_y, sum_z, sum_x2, sum_y2, sum_z2, min_x, min_y, min_z, max_x, max_y, max_z, count = value
    mean_x = sum_x / count
    mean_y = sum_y / count
    mean_z = sum_z / count
    std_x = (sum_x2 / count - mean_x**2)**0.5
    std_y = (sum_y2 / count - mean_y**2)**0.5
    std_z = (sum_z2 / count - mean_z**2)**0.5
    return key, (mean_x, mean_y, mean_z, std_x, std_y, std_z, min_x, min_y, min_z, max_x, max_y, max_z)

def flat_data(data):
    key, value = data
    return key[0], key[1], key[2], value[0], value[1], value[2], value[3], value[4], value[5], value[6], value[7], value[8], value[9], value[10], value[11]

In [77]:
data1 = data1.map(configure_data).reduceByKey(reduce_data).map(calculate_data)
data2 = data2.map(configure_data).reduceByKey(reduce_data).map(calculate_data)
data3 = data3.map(configure_data).reduceByKey(reduce_data).map(calculate_data)
data4 = data4.map(configure_data).reduceByKey(reduce_data).map(calculate_data)

phones = data1.join(data2)
watch = data3.join(data4)

result = phones.union(watch).collect()

In [78]:
for key, value in result:
    print(key, value)

('a', 'gear', 'stand') ((-9.289753201219128, -3.137156524200753, -1.066286300996938, 0.41370414741585154, 1.0628532344784418, 0.6173240991928323, -12.600683, -11.08276, -2.2625206, -0.5650316, -0.5781997, 1.01574), (0.022146662344973242, -0.03286732380555579, -0.07143672518383197, 0.07314944656109791, 0.05570243854351144, 0.050177495900925566, -2.0383835, -0.55287224, -1.2319783, 0.81039995, 0.35446674, 1.1475562))
('a', 'gear', 'null') ((-9.260130103197667, -3.4726260017441857, -1.0407208369767433, 0.031091553476588117, 0.05145894400797844, 0.031430160394143945, -9.3583355, -3.5942953, -1.1420342, -9.173983, -3.3207579, -0.9307459), (0.022570096625655994, -0.036608760294460654, -0.07169493794752187, 0.01779986380982108, 0.011561341072093041, 0.008039675979049308, -0.0314253, -0.08042747, -0.09347696, 0.0561927, -0.018109497, -0.055127434))
('a', 'nexus4', 'stand') ((-6.026499950569997, 0.9334959509016004, 8.013646013120006, 0.184551746735394, 0.24043415892743478, 0.17599985821395306, 

# hay que filtrar los nulos y dejar 24 valores para cada key

('a', 'gear', 'stand') 
(
    (
        -9.289753201219128, -3.137156524200753, -1.066286300996938, 
        0.41370414741585154, 1.0628532344784418, 0.6173240991928323, 
        -12.600683, -11.08276, -2.2625206, 
        -0.5650316, -0.5781997, 1.01574
    ), 
    (
        0.022146662344973242, -0.03286732380555579, -0.07143672518383197, 
        0.07314944656109791, 0.05570243854351144, 0.050177495900925566, 
        -2.0383835, -0.55287224, -1.2319783, 
        0.81039995, 0.35446674, 1.1475562
    )
)
('a', 'gear', 'null') ((-9.260130103197667, -3.4726260017441857, -1.0407208369767433, 0.031091553476588117, 0.05145894400797844, 0.031430160394143945, -9.3583355, -3.5942953, -1.1420342, -9.173983, -3.3207579, -0.9307459), (0.022570096625655994, -0.036608760294460654, -0.07169493794752187, 0.01779986380982108, 0.011561341072093041, 0.008039675979049308, -0.0314253, -0.08042747, -0.09347696, 0.0561927, -0.018109497, -0.055127434))
('a', 'nexus4', 'stand') ((-6.026499950569997, 0.9334959509016004, 8.013646013120006, 0.184551746735394, 0.24043415892743478, 0.17599985821395306, -7.0448303, -0.84251404, 7.149872, -5.5202026, 1.9472808999999998, 8.638794), (0.0015888519490950034, 0.0010094604656469997, 0.00044218442934899877, 0.04277492705495176, 0.028613015987344483, 0.045941044056574176, -0.16569519, -0.15550232, -0.6001586999999999, 0.6321869000000001, 0.34971620000000003, 0.44873047))
('a', 'gear', 'sit') ((-7.604519709183905, -5.5283410786768235, 2.6383265126877022, 0.1855523178504538, 0.27427094510454214, 0.3856803558537603, -10.8229885, -6.9168487, -3.6709096, -6.6588736, -0.83258367, 3.555988), (0.022705615070828633, -0.041422496515184364, -0.06997320145904547, 0.05703368820681714, 0.05626923635200855, 0.04523762117838848, -0.700944, -0.79042625, -0.22849922, 0.39867523, 0.39920786, 0.77604514))
